<a href="https://colab.research.google.com/github/shmanth/Inventory_management_using_Reinforcement_Learning-Q-Learning-/blob/master/IM_env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing necessary libraries
from gym import spaces
import numpy as np

In [ ]:
m=50 # maximum capacity of the warehouse
K=3 #constant cost of order(K in document). it can be cost of petrol
c=4 #variable part of order cost(c(a_t) in the document)
h=0.0025 # holding cost. can be things like rat poison to get rid of rats etc
p=4.5 # selling price such that profit is 12.5%
R=K # Returning cost is same as constant cost as price of petrol is same for to ans fro journies
lamba_mon=16
lamba_tue=31
lamda_wed=15
lamda_thu=32
lamda_fri=30
lamda_sat=8
lamda_sun=42

day_mapping= {0:"mon",1:"tue",2:"wed",3:"thu",4:"fri",5:"sat",6:"sun"}

In [4]:
class InventoryEnv():
  def __init__ (self):
    self.action_space= spaces.Discrete(m+1)
    self.inventory= np.random.choice(np.arange(0,m+1))
    self.day= np.random.choice(0,1,2,3,4,5,6)
    self.state= (self.inventory,self.day)

    # Starting the first round
    self.reset()

  def demand(self,day):
    if day == 0:
      return np.random.poisson(lamda_mon)
    elif day == 1:
      return np.random.poisson(lamda_tue)
    elif day == 2:
      return np.random.poisson(lamda_wed)
    elif day == 3:
      return np.random.poisson(lamda_thu)
    elif day == 4:
      return np.random.poisson(lamda_fri)
    elif day == 5:
      return np.random.poisson(lamda_sat)
    elif day == 6:
      return np.random.poisson(lamda_sun)

  def transition(self, x_t_1, a_t_1, d_t):
    if x_t_1[1] < 6:
      next_day = x_t_1[1] + 1
    else:
      next_day = 0

    stock_after_sales = max(x_t_1[0] - d_t, 0)
    stocks_EOD = min(stock_after_sales + a_t_1, m)

    return(stocks_EOD, next_day)

  def reward(self, x_t_1, a_t_1, d_t):

    # expected income
    expected_income = p * min(x_t_1, d_t)

    # cost of the order
    fixed_order_cost = K * (a_t_1 > 0)
    variable_order_cost = c
    order_cost= fixed_order_cost + variable_order_cost * a_t_1

    #Holding cost
    holding_cost = h * x_t_1

    # opportunity cost
    actual_demand = d_t
    demand_satisfied = x_t_1
    profit = p - c

    opportunity_cost = profit * (actual_demand - demand_satisfied) * (actual_demand > demand_satisfied)
    return r

  def initial_step(self, state, action):
    assert self.action_space.contains(action)  #to check that action is a discrete value less than m
    obs = state

    if state[1] < 6:
      demand = self.demand(state[1]+1)
    else:
      demand = self.demand(0)
    obs2 = self.transition(obs, action, demand) # next state

    return obs2

  def step(self, x_t_1, a_t_1):
    assert self.action_space.contains(ation)

    obs = x_t_1

    if x_t_1[1] < 6:
      d_t = self.demand(x_t_1 + 1)
    else:
      d_t = self.demand(0)

    obs2 = self.transition(obs, a_t_1, d_t) # next state

    reward = self.reward(x_t_1[0], a_t_1, d_t)

    return obs2, reward

  def reset(self):
    return self.state